# 어텐션 메커니즘 (Attention Mechanism)

 ## 기존 seq2seq
 - 인코더에서 입력 시퀀스를 컨텍스트 벡터로 압축하고 디코더가 컨텍스트 벡터를 통해서 출력 시퀀스를 출력
 - 문제점
     * 고정된 크기의 컨텍스트 벡터로 압축하다 보니 정보 손실 발생
     * RNN의 기울기 소실 문제\
 => 입력 문장이 길면 품질 하락

## 어텐션 (Attention) 핵심 아이디어
- 디코더에서 매 시점마다 인코더에서의 전체 입력문장을 다시한번 참고한다
- 단 가장 연관이 있는 입력단어 부분을 좀 더 **집중(attention)**

## 어텐션 함수 (Attention Function)
**Attention(Q, K, V) = Attention Value**\
Q = Query: t시점의 디코더 셀에서의 은닉상태\
K = Keys: 모든 시점의 인코더 셀의 은닉 상태들\
V = Values: 모든 시점의 인코더 셀의 은닉 상태들

![](https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG)

## 닷-프로덕트 어텐션(Dot-Product Attention)
![](https://wikidocs.net/images/page/22893/dotproductattention1_final.PNG)

### 어텐션 스코어 & 어텐션 분포 구하기
- 그림은 디코더 3번째 시점에서 예측하는 과정
- 3번째 시점의 디코더 은닉상태를 s_t, h_t를 인코더의 각 시점의 은닉상태
- s_t와 각 h_t의 내적하여 구한 네개의 값이 어텐션 스코어
- 어텐션 스코어에 softmax를 적용하면 어텐션 분포

### 어텐션 값 or 컨텍스트 벡터 구하기
- 인코더의 은닉상태에 어텐션 분포를 통해 가중합한 값이 어텐션 값 (은닉상태와 같은 모양)

![](https://wikidocs.net/images/page/22893/dotproductattention2_final.PNG)

### 최종 출력
- 어텐션 값과 디코더의 은닉상태를 concatenate하여 s_t를 구함
- 두개의 dense 레이어를 거치는데 활성화 함수로 첫번째는 tanh, 두번째는 softmax를 사용

## 다른 종류의 어텐션
- 다른 어텐션과 닷 프로덕트 어텐션의 차이는 어텐션 스코어 구하는 방법의 차이
![](C:\Users\011\sinjy1203\data\attention.jpg)

## 바다나우 어텐션 (Bahdanau Attention)

### 어텐션 함수 (Attention Function)
**Attention(Q, K, V) = Attention Value**\
Q = Query: t-1 시점의 디코더 셀에서의 은닉 상태\
K = Keys: 모든 시점의 인코더 셀의 은닉 상태들\
V = Values: 모든 시점의 인코더 셀의 은닉 상태들

![](https://wikidocs.net/images/page/73161/%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0%EC%96%B4%ED%85%90%EC%85%981.PNG)

### 어텐션 스코어 (Attention Score) 구하기
$score(s_{t-1}, H) = W_a^T tanh(W_bs_{t-1} + W_cH)$\
s_(t-1): 이전 시점 디코더의 은닉상태\
H: 인코더의 모든 시점 은닉상태 행렬\
W: 학습 가능 가중치\

이후 단계는 닷 프로덕트 어텐션과 동일

### 최종 출력 구하기
- 디코더의 입력에 어텐션값(컨텍스트벡터)를 연결하여 사용
- 이후는 seq2seq와 동일
![](https://wikidocs.net/images/page/73161/%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0%EC%96%B4%ED%85%90%EC%85%985.PNG)

## 양방향 LSTM과 어텐션 메커니즘 (BiLSTM with Attention mechanism

### IMDB 리뷰 데이터 전처리

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 1s 0us/step


In [3]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

### 바다나우 어텐션

In [4]:
import tensorflow as tf

In [5]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)
    
    def call(self, values, query): # values: 인코더 은닉상태들, query: 디코더 이전 시점 은닉상태
        hidden_with_time_axis = tf.expand_dims(query, 1) # shape: (batch, 1, hidden_size)
        
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

### 양방향 LSTM + 어텐션 메커니즘 (BiLSTM with Attention Mechanism)

In [6]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [7]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, 
                              mask_zero=True)(sequence_input)

In [8]:
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)

In [9]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [10]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
98/98 [==============================] - 600s 6s/step - loss: 0.5244 - accuracy: 0.7358 - val_loss: 0.3394 - val_accuracy: 0.8566
Epoch 2/3
98/98 [==============================] - 572s 6s/step - loss: 0.2769 - accuracy: 0.9016 - val_loss: 0.3065 - val_accuracy: 0.8754
Epoch 3/3
98/98 [==============================] - 571s 6s/step - loss: 0.2054 - accuracy: 0.9297 - val_loss: 0.3076 - val_accuracy: 0.8763


In [13]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 375s 478ms/step - loss: 0.3076 - accuracy: 0.8763

 테스트 정확도: 0.8763
